In [2]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [3]:
movies = pd.read_csv("ml-1m/movies.dat", delimiter="::", engine="python", names=["MovieID", "Title", "Genres"])
users = pd.read_csv("ml-1m/users.dat", delimiter="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "ZipCode"])
ratings = pd.read_csv("ml-1m/ratings.dat", delimiter="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"])

movies["Year"] = movies.Title.str[-5:-1]
movies["Year"] = movies["Year"].astype(int)
movies.Title = movies.Title.str[:-7]

In [4]:
db_name = 'movies.db'
con = sqlite3.connect(db_name)

movies.to_sql("movies", con, if_exists='replace', index=False)
users.to_sql("users", con, if_exists='replace', index=False)
ratings.to_sql("ratings", con, if_exists='replace', index=False)

In [5]:
sql_result = pd.read_sql_query("SELECT * FROM movies", con)
sql_result

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000
3879,3949,Requiem for a Dream,Drama,2000
3880,3950,Tigerland,Drama,2000
3881,3951,Two Family House,Drama,2000


In [6]:
sql_result = pd.read_sql_query("SELECT * FROM users", con)
sql_result

,UserID,Gender,Age,Occupation,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [7]:
sql_result = pd.read_sql_query("SELECT * FROM ratings", con)
sql_result

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


# 1. Wybierz wszystkie kolumny (z users)

# 2. Wybierz płeć i wiek użytkowników

# 3. Wybierz pierwszych 5 filmów.

# 4. Wybierz filmy, których ID to nie jest ani 3, ani 5.

* `<>`, `!=`, `AND`
* `IN`, `NOT`

# 5. Wybierz użytkowników, których wiek jest z przedziału [18, 35].

* `AND`, `>=`, `<=`
* `BETWEEN`

# 6. Wybierz użytkowników, których wiek jest z przedziału < 18 lub > 35.

# 7. Wybierz filmy, których rok produkcji jest parzysty, a tytuł filmu zaczyna się na literę T.

In [10]:
sql_result = pd.read_sql_query('SELECT * FROM movies WHERE Year % 2 = 0 AND Title LIKE "T%"', con)
sql_result

,MovieID,Title,Genres,Year
0,64,Two if by Sea,Comedy|Romance,1996
1,111,Taxi Driver,Drama|Thriller,1976
2,306,Three Colors: Red,Drama,1994
3,308,Three Colors: White,Drama,1994
4,326,To Live (Huozhe),Drama,1994
...,...,...,...,...
111,3852,"Tao of Steve, The",Comedy,2000
112,3853,"Tic Code, The",Drama,1998
113,3891,Turn It Up,Crime|Drama,2000
114,3950,Tigerland,Drama,2000


# 8. Wybierz filmy, których rok produkcji jest parzysty lub jest mniejszy od 1995.

In [12]:
sql_result = pd.read_sql_query("""
SELECT *
FROM movies
where year < 1995 OR (Year % 2) = 0
""", con)
sql_result

,MovieID,Title,Genres,Year
0,51,Guardian Angel,Action|Drama|Thriller,1994
1,53,Lamerica,Drama,1994
2,58,"Postino, Il (The Postman)",Drama|Romance,1994
3,61,Eye for an Eye,Drama|Thriller,1996
4,63,Don't Be a Menace to South Central While Drink...,Comedy,1996
...,...,...,...,...
2938,3948,Meet the Parents,Comedy,2000
2939,3949,Requiem for a Dream,Drama,2000
2940,3950,Tigerland,Drama,2000
2941,3951,Two Family House,Drama,2000


# 9. Wybierz unikatowe grupy wiekowe użytkowników.

In [13]:
sql_result = pd.read_sql_query("""
SELECT DISTINCT 
Age, Gender
FROM users
""", con)
sql_result

,Age,Gender
0,1,F
1,56,M
2,25,M
3,45,M
4,50,F
5,35,M
6,35,F
7,25,F
8,50,M
9,18,F


# 10. Wybierz unikatowe kombinacje wieku i płci użytkowników.

# 11. Wybierz zawód użytkownika, gdzie wiek zawiera się w [18, 35], ale ogranicz się do pierwszych 20 wierszy.

# 12. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror

# 13. Wybierz filmy, mieszanka gatunków to Comedy|Romance, Action|Crime|Thriller lub Comedy|Horror, a rok produkcji jest większy niż 1985.

In [15]:
sql_result = pd.read_sql_query("""
SELECT  
*
FROM movies
WHERE YEAR>1985 AND genres IN ('Comedy|Romance', 'Action|Crime|Thriller', 'Comedy|Horror')
""", con)
sql_result

,MovieID,Title,Genres,Year
0,3,Grumpier Old Men,Comedy|Romance,1995
1,6,Heat,Action|Crime|Thriller,1995
2,7,Sabrina,Comedy|Romance,1995
3,12,Dracula: Dead and Loving It,Comedy|Horror,1995
4,39,Clueless,Comedy|Romance,1995
...,...,...,...,...
143,3785,Scary Movie,Comedy|Horror,2000
144,3829,Mad About Mambo,Comedy|Romance,2000
145,3874,"Couch in New York, A",Comedy|Romance,1996
146,3885,Love & Sex,Comedy|Romance,2000


# Sortowanie

# 1. Posortuj filmy po tytule.

# 2. Posortuj filmy po tytule, ale tylko te, których tytuł zaczyna się na literę T

In [19]:
UNION

sql_result = pd.read_sql_query("""
SELECT Title
FROM movies
where title like "T%"
ORDER BY TITLE
""", con)
sql_result

,Title
0,T-Men
1,Taffin
2,Tainted
3,Take the Money and Run
4,"Taking of Pelham One Two Three, The"
...,...
228,Two Much
229,Two Thousand Maniacs!
230,Two Women (La Ciociara)
231,Two if by Sea


In [24]:
sql_result = pd.read_sql_query("""
SELECT title
FROM movies
ORDER BY case when title like 'T%' then title else NULL end
""", con)
sql_result

,Title
0,Jumanji
1,Grumpier Old Men
2,Waiting to Exhale
3,Father of the Bride Part II
4,Heat
...,...
3878,Two Much
3879,Two Thousand Maniacs!
3880,Two Women (La Ciociara)
3881,Two if by Sea


# 3. Wybierz unikatowe grupy wiekowe użytkowników i posortuj.

# 4. Posortuj filmy po roku produkcji malejąco, a w ramach jednego roku po tytule rosnąco.

# 5. Wybierz unikatowe kombinacje wieku i płci użytkowników i posortuj, najpierw po płci, a potem po wieku.

# 6. Wybierz unikatowe kombinacje wieku i płci użytkowników, wybierz tylko pełnoletnie kobiety i posortuj malejąco po wieku.

In [26]:
sql_result = pd.read_sql_query("""
SELECT  DISTINCT
Age, Gender
FROM users
WHERE Age>=18 AND Gender = "F"
ORDER BY Age DESC
""", con)
sql_result

,Age,Gender
0,56,F
1,50,F
2,45,F
3,35,F
4,25,F
5,18,F


# Grupowanie

# 1. Wybierz liczbę filmów o danej mieszaninie gatunków.

przykładowa mieszanina: `Comedy|Romance`

# 2. Wybierz unikatowe kombinacje wieku i płci użytkowników, zlicz, ile jest rekordów dla każdej kombinacji i posortuj, najpierw po płci, a potem po wieku(a osobno po liczbie reprezentantów każdej grupy).

In [27]:
sql_result = pd.read_sql_query("""
SELECT  Age, Gender, COUNT(*)
from users
GROUP BY Age, Gender
ORDER BY Gender, Age

""", con)
sql_result

,Age,Gender,COUNT(*)
0,1,F,78
1,18,F,298
2,25,F,558
3,35,F,338
4,45,F,189
5,50,F,146
6,56,F,102
7,1,M,144
8,18,M,805
9,25,M,1538


In [28]:
sql_result = pd.read_sql_query("""
SELECT  Age, Gender, COUNT(*)
from users
GROUP BY Age, Gender
ORDER BY COUNT(*) 

""", con)
sql_result

,Age,Gender,COUNT(*)
0,1,F,78
1,56,F,102
2,1,M,144
3,50,F,146
4,45,F,189
5,56,M,278
6,18,F,298
7,35,F,338
8,50,M,350
9,45,M,361


# 3. Wybierz minimalną, maksymalną i średnią ocenę dla każdego filmu (bez join).

# 4. Ile filmów z literą X w tytule pojawiało się w poszczególnych latach?

# 5. Wybierz lata i liczbę filmów, których tytuły zawierają literę X, ale tylko takie lata, że takich filmów było więcej niż 3.

# 6. Wybierz lata i liczbę filmów i posortuj od najpłodniejszego roku do najmniej, zostaw jedynie pierwszych 10 wierszy.

# Join

# 1. Połącz ratings z movies

# 2. Jaki jest najlepszy film wszechczasów?

* albo join i potem groupby
* albo zrobić podzapytanie które robi groupby i dopiero potem join

# 3. Bierzemy pod uwagę filmy ze >= 100 głosami

# 4. Wyświetl pełną nazwę filmu, który miał najwięcej głosów